# Explore hologram data Repeatability on PWV : Update March 2025 about selection

- author Sylvie Dagoret-Campagne
- creation date 2025-09-16 : version v1
- affiliation : IJCLab
- Kernel @usdf **w_2025_36*
- Home emac : base (conda)
- laptop : conda_py310

**Goal** : Show Night variations of PWV wrt date and Time. Fit a straight line.

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import os

In [ ]:
# where are stored the figures
pathfigs = "figs_PWV02_PWVRepeatability"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from astropy.coordinates.earth import EarthLocation
from datetime import datetime
from pytz import timezone

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import scipy
from scipy.optimize import curve_fit,least_squares


# new color correction model
import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
# Remove to run faster the notebook
#import ipywidgets as widgets
#%matplotlib widget

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
pd.__version__

### quartiles

In [ ]:
def q50(x):
    return x.quantile(0.5)

# 90th Percentile
def q90(x):
    return x.quantile(0.9)

def q25(x):
    return x.quantile(0.25)

def q75(x):
    return x.quantile(0.75)

def iqr(x):
    irq =  q75(x)-q25(x)
    return irq

#https://en.wikipedia.org/wiki/Interquartile_range    
#SD = IQR/1.35. for a normal distribution
def std_iqr(x):
    return iqr(x)/1.349
    
#df.groupby("AGGREGATE").quantile([0, 0.25, 0.5, 0.75, 0.95, 1])
#df.groupby("AGGREGATE").agg(("YOUR_COL_NAME", lambda x: x.quantile(0.5))

In [ ]:
1.35/2

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

### Configuration

In [ ]:
observing_location = EarthLocation.of_site('Rubin Observatory')
tz = timezone('America/Santiago')

In [ ]:
FLAG_WITHCOLLIMATOR = True
DATE_WITHCOLLIMATOR = 20230930
datetime_WITHCOLLIMATOR = convertNumToDatestr(DATE_WITHCOLLIMATOR)
datetime_WITHCOLLIMATOR = pd.to_datetime("2023-09-30 00:00:00.0+0000")
datetime_WITHCOLLIMATOR

In [ ]:
version_results = "v1"
legendtag = {"v1" : "v3.1.0"}

In [ ]:
atmfilenamesdict = {"v1":"../2025-06-26-SpectractorExtraction-FromButler/data/spectro/auxtel_atmosphere_20250912a_repomain_v1.npy",}

In [ ]:
atmfilename = atmfilenamesdict[version_results]
tag = legendtag[version_results] 

## Initialisation

### Read the file

In [ ]:
specdata = np.load(atmfilename,allow_pickle=True)

In [ ]:
df_spec = pd.DataFrame(specdata)

In [ ]:
#list(df_spec.columns)

### Compute NightObs

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
if FLAG_WITHCOLLIMATOR:
    df_spec = df_spec[df_spec["nightObs"]> DATE_WITHCOLLIMATOR]
    

#### Series on spec

In [ ]:
ser_spec_size = df_spec.groupby(["nightObs"]).size()

In [ ]:
fig,ax = plt.subplots(1,1)
ax.hist(ser_spec_size.values,bins=50,facecolor="b")
ax.set_title("nb obs per night")
ax.set_xlabel("Nobs/night")

##### Make 3 series

In [ ]:
ser_CHI2_FIT = df_spec[["CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])
ser_PWV = df_spec[["PWV [mm]_x","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])
ser_PWV_CHI2_FIT = df_spec[["PWV [mm]_x","CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])

In [ ]:
#ser_PWV

In [ ]:
#ser_CHI2_FIT

In [ ]:
#ser_PWV_CHI2_FIT 

### Plot PWV and Chi2 from sereis before any selection

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV.unstack()["PWV [mm]_x"]["count"].plot(kind='bar', ax=ax,subplots=False, rot=90,figsize=(18,4),facecolor="b",grid=True,title="Number of measurements per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV.unstack()["PWV [mm]_x"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Mean PWV per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV.unstack()["PWV [mm]_x"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Median PWV per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV.unstack()["PWV [mm]_x"]["std"].plot(kind='bar', subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="STD variation for PWV per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["count"].plot(kind='bar', ax=ax,subplots=False, rot=90,figsize=(18,4),facecolor="r",grid=True,title="Number of measurements per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Mean CHI2 per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Median CHI2 per night")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT.unstack()["CHI2_FIT"]["std"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="STD variation CHI2 per night")
plt.tight_layout()

In [ ]:
#ser_PWV_CHI2_FIT 

In [ ]:
ser_PWV_CHI2_FIT.loc[20231010,:]

In [ ]:
df_unstack = ser_PWV_CHI2_FIT.loc[20231010,:].unstack()
df_unstack

In [ ]:
ser_PWV_CHI2_FIT.loc[20231010,"PWV [mm]_x"]["count"]

In [ ]:
df_unstack.loc["PWV [mm]_x","count"]

#### Add aggregate data added to pandas dataframe

In [ ]:
def FillAgreggates(row):
    the_nightObs = row["nightObs"]
    df_night = ser_PWV_CHI2_FIT.loc[the_nightObs,:].unstack()
    count = df_night.loc["PWV [mm]_x","count"]
    pwvmin = df_night.loc["PWV [mm]_x","min"]
    pwvmax = df_night.loc["PWV [mm]_x","max"]
    pwvmean = df_night.loc["PWV [mm]_x","mean"]
    pwvmedian = df_night.loc["PWV [mm]_x","median"]
    pwvstd = df_night.loc["PWV [mm]_x","std"]
    chi2min = df_night.loc["CHI2_FIT","min"]
    chi2max = df_night.loc["CHI2_FIT","max"]
    chi2mean = df_night.loc["CHI2_FIT","mean"]
    chi2median = df_night.loc["CHI2_FIT","median"]
    chi2std = df_night.loc["CHI2_FIT","std"]
    
    d = {"_count":count,"_pwvmin":pwvmin,"_pwvmax":pwvmax,"_pwvmean":pwvmean,"_pwvmedian":pwvmedian,"_pwvstd":pwvstd,
        "_chi2min":chi2min,"_chi2max":chi2max,"_chi2mean":chi2mean,"_chi2median":chi2median,"_chi2std":chi2std}
    
    ser = pd.Series(data=d)

    return ser

In [ ]:
row = df_spec.iloc[0]

In [ ]:
row

In [ ]:
FillAgreggates(row)

#### join dataframe + aggregates

In [ ]:
df_spec = df_spec.join(df_spec.apply(FillAgreggates,axis=1,result_type="expand"))

## Apply Quality selection

In [ ]:
fig,ax = plt.subplots(1,1)
df_spec["CHI2_FIT"].hist(bins=50,ax=ax,range=(0,200))
ax.set_yscale("log")

### Add the Time in pd.datetime

In [ ]:
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])

## Compute night boundaries

In [ ]:
def GetNightBoundariesDict(df_spec):
    """
    input:
      df_spec the dataframe for spectroscopy summary results
    output:
      the dict of night boudaries
    """
    
    Dt = pd.Timedelta(minutes=30)
    d = {}
    list_of_nightobs = df_spec["nightObs"].unique()
    for nightobs in list_of_nightobs:
        sel_flag = df_spec["nightObs"]== nightobs
        df_night = df_spec[sel_flag]
        tmin = df_night["Time"].min()-Dt
        tmax = df_night["Time"].max()+Dt
        d[nightobs] = (tmin,tmax)
    return d

In [ ]:
dn = GetNightBoundariesDict(df_spec)

## Plot all data

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax  = axs
leg=ax.get_legend()

df_spec.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg)
ax.set_ylabel("PWV [mm]_x")

ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Precipitable water vapor measured by holo vs time")

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

plt.tight_layout()

## Apply Quality selection cuts

In [ ]:
def getSelectionCutVeryOld(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9):
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) 
    return cut

In [ ]:
def getSelectionCutOld(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.):
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    (df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) & \
    (df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) 
    return cut

In [ ]:
def getSelectionCutOld2(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.,vaodmax=0.1):
    #cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.75) &  (df_spec["D2CCD"]<187.75) &  (df_spec["VAOD_x"]<0.3) & \
    (df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) & \
    (df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) 
    return cut

In [ ]:
#def getSelectionCut(df_spec, chi2max=20., pwvmin=0.1, pwvmax = 14.9,ozmin=100.,ozmax=600.):
def getSelectionCut(df_spec, chi2max=200., pwvmin=0, pwvmax = 16,ozmin=0.,ozmax=650.):
    #cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.5) &  (df_spec["D2CCD"]<187.3) & \
    cut =  (df_spec["CHI2_FIT"]<chi2max) & (df_spec["PWV [mm]_x"] > pwvmin) & (df_spec["PWV [mm]_x"] < pwvmax) & (df_spec["D2CCD"]>186.0) &  (df_spec["D2CCD"]<189.0) & \
    (df_spec['EXPTIME'] > 20.) & (df_spec["PWV [mm]_y"] > pwvmin) & (df_spec["PWV [mm]_y"] < pwvmax) 
    #(df_spec["ozone [db]_y"] > ozmin) & (df_spec["ozone [db]_y"] < ozmax) & (df_spec["TARGET"] == "HD185975")
    return cut

In [ ]:
cut = getSelectionCut(df_spec)

In [ ]:
df_spec_sel = df_spec[cut].drop(labels=['_count', '_pwvmin', '_pwvmax', '_pwvmean', '_pwvmedian', '_pwvstd','_chi2min', '_chi2max', '_chi2mean', '_chi2median', '_chi2std'],axis=1)

In [ ]:
df_spec_sel.reset_index(drop=True,inplace=True)

## Compute per-night aggregates

### Compute series per night

In [ ]:
ser_PWV_CHI2_FIT_sel = df_spec_sel[["PWV [mm]_x","CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])

In [ ]:
def FillAgreggatesSel(row):
    the_nightObs = row["nightObs"]
    df_night = ser_PWV_CHI2_FIT_sel.loc[the_nightObs,:].unstack()
    count = df_night.loc["PWV [mm]_x","count"]
    pwvmin = df_night.loc["PWV [mm]_x","min"]
    pwvmax = df_night.loc["PWV [mm]_x","max"]
    pwvmean = df_night.loc["PWV [mm]_x","mean"]
    pwvmedian = df_night.loc["PWV [mm]_x","median"]
    pwvstd = df_night.loc["PWV [mm]_x","std"]
    chi2min = df_night.loc["CHI2_FIT","min"]
    chi2max = df_night.loc["CHI2_FIT","max"]
    chi2mean = df_night.loc["CHI2_FIT","mean"]
    chi2median = df_night.loc["CHI2_FIT","median"]
    chi2std = df_night.loc["CHI2_FIT","std"]
    
    d = {"_count":count,"_pwvmin":pwvmin,"_pwvmax":pwvmax,"_pwvmean":pwvmean,"_pwvmedian":pwvmedian,"_pwvstd":pwvstd,
        "_chi2min":chi2min,"_chi2max":chi2max,"_chi2mean":chi2mean,"_chi2median":chi2median,"_chi2std":chi2std}
    
    ser = pd.Series(data=d)

    return ser
    

In [ ]:
df_spec_sel = df_spec_sel.join(df_spec_sel.apply(FillAgreggatesSel,axis=1,result_type="expand"))

In [ ]:
TMIN_sel = df_spec_sel["nightObs"].min()
TMAX_sel = df_spec_sel["nightObs"].max()

## Recompute night boundaries

In [ ]:
dn = GetNightBoundariesDict(df_spec_sel)

## Plot all data

In [ ]:
from matplotlib.dates import DateFormatter
date_form = DateFormatter("%y-%m-%dT%H:%M")
fig,axs = plt.subplots(1,1,figsize=(14,6))
ax  = axs
leg=ax.get_legend()

df_spec_sel.plot(x="Time",y="PWV [mm]_x",ax=ax,marker='+',c="r",lw=0.0,grid=True,label=tag,legend=leg,ms=10)
ax.set_ylabel("PWV [mm]_x")
ax.set_xlabel("time")
ax.xaxis.set_major_formatter(date_form)
ax.set_title("Precipitable water vapor measured by holo selected vs time")

for key, tt in dn.items():
    ax.axvspan(tt[0],tt[1], color='blue', alpha=0.1)

plt.tight_layout()
figname =f"{pathfigs}/pwv_allpoints_allnights"+figtype
plt.savefig(figname)
plt.show()

## Plot series on selected data

In [ ]:
ser_CHI2_FIT_sel = df_spec_sel[["CHI2_FIT","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])
ser_PWV_sel = df_spec_sel[["PWV [mm]_x","nightObs"]].groupby(["nightObs"]).agg(['count','min', 'max','mean','std','median'])

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV_sel.unstack()["PWV [mm]_x"]["count"].plot(kind='bar', ax=ax,subplots=False, rot=90,figsize=(18,4),facecolor="b",grid=True,title="Number of measurements per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV_sel.unstack()["PWV [mm]_x"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Mean PWV per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV_sel.unstack()["PWV [mm]_x"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="Median PWV per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_PWV_sel.unstack()["PWV [mm]_x"]["std"].plot(kind='bar', subplots=False, rot=90,figsize=(18,4),facecolor='b',grid=True,title="STD variation for PWV per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT_sel.unstack()["CHI2_FIT"]["mean"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Mean CHI2 per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT_sel.unstack()["CHI2_FIT"]["median"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="Median CHI2 per night after selection")
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(18,3))
ser_CHI2_FIT_sel.unstack()["CHI2_FIT"]["std"].plot(kind='bar',ax=ax ,subplots=False, rot=90,figsize=(18,4),facecolor='r',grid=True,title="STD variation CHI2 per night after selection")
plt.tight_layout()

## Show time variation each night

In [ ]:
all_selected_nights = df_spec_sel["nightObs"].unique()

In [ ]:
def funclineres(params, x, y, yerr):
    # Return residual = fit-observed
    return (y-params[0] -params[1]*x)/yerr
def funcline(params,x):
    return params[0] + params[1]*x

In [ ]:
def MakeLineFit(df_night_pwv_curve):
    """
    """

    x = df_night_pwv_curve["dt"].values
    y = df_night_pwv_curve["PWV [mm]_x"].values
    yerr = df_night_pwv_curve["PWV [mm]_err_x"].values
    n = len(y)
        
    #popt, pcov = optimize.curve_fit(f, x, y, [1,-4])
    fit_res = least_squares(funclineres,[5.,0],args = (x,y,yerr))
    popt = fit_res.x 
    npar = len(popt)
    J = fit_res.jac
    cov = np.linalg.inv(J.T.dot(J))
    chi2dof = ((funclineres(popt,x,y,yerr))**2).sum()/(n-npar)
    cov *= chi2dof
    perr = np.sqrt(np.diagonal(cov)) 
    
    xfit = np.linspace(x.min()*0.99,x.max()*1.05)
    yfit = funcline(popt,xfit)
        
    slope = popt[1]
    slope_err = perr[1]

    return x,y,yerr,n,chi2dof,xfit,yfit,slope,slope_err

### Plot night by night

In [ ]:
df_spec_night = df_spec_sel[df_spec_sel["nightObs"] == 20240923]
#select the variables
df_night_pwv_curve = df_spec_night[["nightObs","Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","TARGET"]]
tmin = df_night_pwv_curve["Time"].min()
df_night_pwv_curve["dt"] = (df_night_pwv_curve["Time"] - tmin).dt.total_seconds()/3600.

In [ ]:
df_night_pwv_curve.iloc[0]

In [ ]:
def ComputeRepeatability(df):
    """
    Compute Repeatability of PWV for Spectrogram and and Spectrum
    
    """
    N = len(df)
    dfout = pd.DataFrame(index=df.index,columns = ["nightObs","dt","dt_rep","dPWVx_rep","dPWVy_rep","targflag_rep","Npoints"])
    dfout["targflag_rep"].astype(bool)
    #dfout["Npoints"].astype(int)
    
    target_old = "No"
    time_old = 0.
    pwvx_old = 0.
    pwvy_old = 0.
    
    for index in range(N):
        
        nightObs =  df.iloc[index]["nightObs"]
        
        if index ==0:
            dt0 = df.iloc[index]["dt"]
            dfout.iloc[index] = [ nightObs,dt0, 0., 0., 0., False,N]
        else:
            target_new = df.iloc[index]["TARGET"]
            time_new = df.iloc[index]["dt"]
            pwvx_new = df.iloc[index]["PWV [mm]_x"]
            pwvy_new = df.iloc[index]["PWV [mm]_y"]
            
            flag_target = (target_new == target_old)
            dPWVx_rep = pwvx_new - pwvx_old
            dPWVy_rep = pwvy_new - pwvy_old
            dt_rep = (time_new-time_old)*3600. # in seconds
            
            dfout.iloc[index] = [ nightObs,time_new, dt_rep, dPWVx_rep, dPWVy_rep, flag_target,N]
        
        target_old = df.iloc[index]["TARGET"]
        time_old = df.iloc[index]["dt"]
        pwvx_old = df.iloc[index]["PWV [mm]_x"]
        pwvy_old = df.iloc[index]["PWV [mm]_y"]
        
    return dfout

In [ ]:
ComputeRepeatability(df_night_pwv_curve).head()

In [ ]:
all_df_repeatability = []

# loop on nights
for night in all_selected_nights:
    # Select the night
    df_spec_night = df_spec_sel[df_spec_sel["nightObs"] == night]
    
    # Select the variables
    df_night_pwv_curve = df_spec_night[["nightObs","Time","PWV [mm]_x","PWV [mm]_err_x","PWV [mm]_y","PWV [mm]_err_y","TARGET"]]

    tmin = df_night_pwv_curve["Time"].min()

    # Convert in hours
    df_night_pwv_curve["dt"] = (df_night_pwv_curve["Time"] - tmin).dt.total_seconds()/3600.
    
    # Compute the repeatability on PWV x and y
    df_rep = ComputeRepeatability(df_night_pwv_curve)
    
    # Keep all repeatability
    all_df_repeatability.append(df_rep)
    
    cut_on_dt = (df_rep["dt_rep"]>0.) & (df_rep["dt_rep"]< 120.)   
    cut_on_target = df_rep["targflag_rep"]
    
    cut_loose = cut_on_dt
    cut_strong = cut_on_dt & cut_on_target 
        
    # plot  
    fig,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(12,4))
    
    leg1 = ax1.get_legend()
    leg2 = ax2.get_legend()
    
      
    df_rep[cut_strong]["dPWVx_rep"].hist(ax=ax1,bins=60,range=(-3,3),facecolor="b",legend=leg1,label="Same target")
    df_rep[cut_strong]["dPWVy_rep"].hist(ax=ax2,bins=60,range=(-3,3),facecolor="b")
    
    df_rep[cut_loose]["dPWVx_rep"].hist(ax=ax1,bins=60,range=(-3,3),color="r",lw=3,histtype="step")
    df_rep[cut_loose]["dPWVy_rep"].hist(ax=ax2,bins=60,range=(-3,3),color="r",lw=3,histtype="step")
    
    ax1.set_xlabel("PWV (mm)")
    ax1.set_title("Spectrogram")
    ax2.set_xlabel("PWV (mm)")
    ax2.set_title("1D-Spectrum")
    
    df_rep[cut_strong].plot.scatter(x="dPWVx_rep",y="dPWVy_rep",marker='.',c="b",ax=ax3)
    ax3.set_xlim(-3,3.)
    ax3.set_ylim(-3,3.)
    ax3.grid()
    ax3.set_xlabel("PWV (2D) (mm)")
    ax3.set_ylabel("PWV (1D) (mm)")
    
    
    title = f"PWV repeatability for Night {night}"
    plt.suptitle(title) 
    plt.tight_layout()
    #figname =f"{pathfigs}/pwv_per_night_{night}"+figtype
    #    plt.savefig(figname)
    plt.show()

       

## Merge all repeatability dataFrame

In [ ]:
df_allrep = pd.concat(all_df_repeatability)

In [ ]:
df_allrep.head()

## Selection

In [ ]:
cut_on_dt = (df_allrep["dt_rep"]>0.) & (df_allrep["dt_rep"]< 120.) & (df_allrep["Npoints"] > 10.)     
cut_on_target = df_allrep["targflag_rep"]
    
cut_loose = cut_on_dt
cut_strong = cut_on_dt & cut_on_target 

In [ ]:
dfl = df_allrep[cut_loose][["dt","dt_rep","dPWVx_rep","dPWVy_rep"]]
dfs = df_allrep[cut_strong][["dt","dt_rep","dPWVx_rep","dPWVy_rep"]]

### Compute Statistics

- apply statistics on strong criteria

In [ ]:
#dfl_stat = dfl.aggregate(["count","mean","std"])
dfl_stat = dfl.aggregate(["count","mean","std",lambda x : std_iqr(x)])

In [ ]:
dfs_stat = dfl.aggregate(["count","mean","std",lambda x : std_iqr(x)])

In [ ]:
dfs_stat

In [ ]:
# strong cut
#dfs_stat = dfs.aggregate(["count","mean","std"])
dfs_stat = dfl.aggregate(["count","mean","std",lambda x : std_iqr(x)])
meanx = dfs_stat.loc["mean","dPWVx_rep"]
stdx = dfs_stat.loc["std","dPWVx_rep"]
stdx_iqr = dfs_stat.loc["<lambda>","dPWVx_rep"]

meany = dfs_stat.loc["mean","dPWVy_rep"]
stdy = dfs_stat.loc["std","dPWVy_rep"]
stdy_iqr = dfs_stat.loc["<lambda>","dPWVy_rep"]

dfs["ddPWVx_rep"] = np.abs(dfs["dPWVx_rep"] - meanx)
dfs["ddPWVy_rep"] = np.abs(dfs["dPWVy_rep"] - meany)

dfs_stat = dfs.aggregate(["count","mean","std"])

meanx = dfs_stat.loc["mean","dPWVx_rep"]
stdx = dfs_stat.loc["std","dPWVx_rep"]
madx =  dfs_stat.loc["mean","ddPWVx_rep"]
sigma_madx = np.sqrt(np.pi/2.)*madx

meany = dfs_stat.loc["mean","dPWVy_rep"]
stdy = dfs_stat.loc["std","dPWVy_rep"]
mady =  dfs_stat.loc["mean","ddPWVy_rep"]
sigma_mady = np.sqrt(np.pi/2.)*mady

textstr_pwvx = "$\delta_{pwv}$ = "+ f"{meanx:.2f}" + "$\pm + $" + f"{stdx:.2f} mm" + "\n" + "$\\sigma_{MAD} = $" + f" {sigma_madx:.2f} mm"
textstr_pwvy = "$\delta_{pwv}$ = "+ f"{meany:.2f}" + "$\pm + $" + f"{stdy:.2f} mm" + "\n" + "$\\sigma_{MAD} = $" + f" {sigma_mady:.2f} mm"

In [ ]:
# strong cut
#dfs_stat = dfs.aggregate(["count","mean","std"])
dfs_stat = dfs.aggregate(["count","mean","std",lambda x : std_iqr(x)])
meanx = dfs_stat.loc["mean","dPWVx_rep"]
stdx = dfs_stat.loc["std","dPWVx_rep"]
stdx_iqr = dfs_stat.loc["<lambda>","dPWVx_rep"]

meany = dfs_stat.loc["mean","dPWVy_rep"]
stdy = dfs_stat.loc["std","dPWVy_rep"]
stdy_iqr = dfs_stat.loc["<lambda>","dPWVy_rep"]

dfs["ddPWVx_rep"] = np.abs(dfs["dPWVx_rep"] - meanx)
dfs["ddPWVy_rep"] = np.abs(dfs["dPWVy_rep"] - meany)

#dfs_stat = dfs.aggregate(["count","mean","std"])
dfs_stat = dfs.aggregate(["count","mean","std",lambda x : std_iqr(x)])

meanx = dfs_stat.loc["mean","dPWVx_rep"]
stdx = dfs_stat.loc["std","dPWVx_rep"]
stdx_iqr = dfs_stat.loc["<lambda>","dPWVx_rep"]
madx =  dfs_stat.loc["mean","ddPWVx_rep"]
sigma_madx = np.sqrt(np.pi/2.)*madx

meany = dfs_stat.loc["mean","dPWVy_rep"]
stdy = dfs_stat.loc["std","dPWVy_rep"]
stdy_iqr = dfs_stat.loc["<lambda>","dPWVy_rep"]
mady =  dfs_stat.loc["mean","ddPWVy_rep"]
sigma_mady = np.sqrt(np.pi/2.)*mady

textstr_pwvx = "$\delta_{pwv}$ = "+ f"{meanx:.2f}" + "$\pm + $" + f"{stdx:.2f} mm" + "\n" + "$\\sigma_{MAD} = $" + f" {sigma_madx:.2f} mm"
textstr_pwvy = "$\delta_{pwv}$ = "+ f"{meany:.2f}" + "$\pm + $" + f"{stdy:.2f} mm" + "\n" + "$\\sigma_{MAD} = $" + f" {sigma_mady:.2f} mm"

In [ ]:
textstr_pwvx = "$\delta_{pwv}$ = "+ f"{meanx:.2f}" + "$\pm + $" + f"{stdx:.2f} mm" + "\n" + "$\\sigma_{MAD} = $" + f" {sigma_madx:.2f} mm" + "\n" + "$RMS = $" + f" {stdx:.2f} mm" + "\n" + "$\sigma_{IQR} = $" + f" {stdx_iqr:.2f} mm"
textstr_pwvy = "$\delta_{pwv}$ = "+ f"{meany:.2f}" + "$\pm + $" + f"{stdy:.2f} mm" + "\n" + "$\\sigma_{MAD} = $" + f" {sigma_mady:.2f} mm" + "\n" + "$RMS = $" + f" {stdy:.2f} mm"

In [ ]:
dfs_stat

In [ ]:
dfl_stat

## Plot repeatability

In [ ]:
# plot  
fig,ax1 = plt.subplots(1,1,figsize=(5,4),layout='constrained')
    
leg1 = ax1.get_legend()
     
df_allrep[cut_strong]["dPWVx_rep"].hist(ax=ax1,bins=60,range=(-3,3),facecolor="b",legend=leg1,label="Same target") 
df_allrep[cut_loose]["dPWVx_rep"].hist(ax=ax1,bins=60,range=(-3,3),color="r",lw=3,histtype="step")

ax1.set_xlabel("$\Delta$ PWV (mm)")

title = f"PWV repetability ({TMIN_sel} - {TMAX_sel})"

ax1.set_title(title,fontsize=12)
ax1.text(0.05, 0.95, textstr_pwvx, transform=ax1.transAxes, fontsize=14,verticalalignment='top', bbox=props)
#ax1.legend()

figname =f"{pathfigs}/pwv_repeatability_allnights_single"+figtype
plt.savefig(figname)
plt.show()